<a href="https://colab.research.google.com/github/wesamhamad/Base-Arm_ControlPanel/blob/main/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **10 points as iput**

**Naïve version**

In [ ]:
import math

# Haversine distance function
def haversine(coord1, coord2):
    lat1, lon1 = math.radians(coord1[0]), math.radians(coord1[1])
    lat2, lon2 = math.radians(coord2[0]), math.radians(coord2[1])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    R = 6371  # Radius of Earth in kilometers
    return R * c

# Naive nearest neighbor TSP solver
def naive_nearest_neighbor_tsp(points):
    n = len(points)
    min_path = None
    min_distance = float('inf')

    def calculate_path_distance(path):
        distance = 0
        for i in range(len(path) - 1):
            distance += haversine(points[path[i]], points[path[i + 1]])
        # Add distance from last to first for a complete tour
        distance += haversine(points[path[-1]], points[path[0]])
        return distance

    def generate_all_paths(current_path, remaining_points):
        nonlocal min_path, min_distance
        if not remaining_points:
            current_distance = calculate_path_distance(current_path)
            if current_distance < min_distance:
                min_distance = current_distance
                min_path = current_path[:]
            return

        for i in range(len(remaining_points)):
            next_point = remaining_points[i]
            generate_all_paths(current_path + [next_point], remaining_points[:i] + remaining_points[i+1:])

    generate_all_paths([0], list(range(1, n)))

    return min_path, min_distance

# Qassim region cities GPS coordinates
# points = [
#     (26.32599, 43.97497),  # Buraidah
#     (26.08427, 43.99355),  # Unaizah
#     (25.86944, 43.49731),  # Ar Rass
#     (26.14422, 43.65986),  # Al Bukayriyah
# ]
points = [
    (26.32599, 43.97497),  # Buraidah
    (26.08427, 43.99355),  # Unaizah
    (25.86944, 43.49731),  # Ar Rass
    (26.14422, 43.65986),  # Al Bukayriyah
    (26.29945, 43.77493),  # Al Badayea
    (25.79886, 43.52471),  # Al Mithnab
    (26.12918, 43.64305),  # Al Rass
    (26.08748, 43.76251),  # An Nabhaniyah
    (26.07385, 44.30558),  # Badai
    (26.16556, 43.70222),  # Mulida
]


# Use the naive TSP solver with the points
tour, total_distance = naive_nearest_neighbor_tsp(points)

# Print the tour and total distance
print("Tour:", tour)
print("Total Distance (km):", total_distance)


Tour: [0, 8, 1, 7, 5, 2, 6, 3, 9, 4]
Total Distance (km): 222.08758743046747


 Both our implementation and the original brute-force algorithm explore all possible paths to find the shortest one, the way they generate and compare these paths differs. Our method uses a recursive strategy to build and evaluate paths incrementally, whereas the classic brute-force method generates all permutations upfront and then evaluates them.



>

**Optimized Nearest Neighbor Heuristic Using a Priority Queue (As Naive)**

In [ ]:
import math
import heapq

def haversine(coord1, coord2):
    """Calculate the great-circle distance between two points on the Earth."""
    lat1, lon1 = math.radians(coord1[0]), math.radians(coord1[1])
    lat2, lon2 = math.radians(coord2[0]), math.radians(coord2[1])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    R = 6371  # Radius of Earth in kilometers
    return R * c

def optimized_nearest_neighbor_tsp(points):
    """Apply an optimized Nearest Neighbor Heuristic using a priority queue."""
    total_distance = 0
    n = len(points)
    tour = [0]
    visited = set([0])
    min_heap = [(haversine(points[0], points[i]), i) for i in range(1, n)]
    heapq.heapify(min_heap)

    while len(visited) < n:
        _, next_node = heapq.heappop(min_heap)
        while next_node in visited:
            _, next_node = heapq.heappop(min_heap)
        total_distance += haversine(points[tour[-1]], points[next_node])
        tour.append(next_node)
        visited.add(next_node)
        for i in range(n):
            if i not in visited:
                heapq.heappush(min_heap, (haversine(points[next_node], points[i]), i))

    total_distance += haversine(points[tour[-1]], points[tour[0]])  # Return to start
    return tour, total_distance

# Qassim region cities GPS coordinates
points = [
    (26.32599, 43.97497),  # Buraidah
    (26.08427, 43.99355),  # Unaizah
    (25.86944, 43.49731),  # Ar Rass
    (26.14422, 43.65986),  # Al Bukayriyah
    (26.29945, 43.77493),  # Al Badayea
    (25.79886, 43.52471),  # Al Mithnab
    (26.12918, 43.64305),  # Al Rass
    (26.08748, 43.76251),  # An Nabhaniyah
    (26.07385, 44.30558),  # Badai
    (26.16556, 43.70222),  # Mulida
]


tour, total_distance = optimized_nearest_neighbor_tsp(points)

# Print the tour and total distance
print("Tour:", tour)
print("Total Distance (km):", total_distance)


Tour: [0, 4, 9, 3, 6, 7, 1, 8, 2, 5]
Total Distance (km): 277.122155632146


**A star implementation for TSP**

In [ ]:
import math
import heapq

def haversine(coord1, coord2):
    """Calculate the great-circle distance between two points on the Earth."""
    lat1, lon1 = math.radians(coord1[0]), math.radians(coord1[1])
    lat2, lon2 = math.radians(coord2[0]), math.radians(coord2[1])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    R = 6371  # Radius of Earth in kilometers
    return R * c

def heuristic(node, points, unvisited, start_point):
    if not unvisited:
        return haversine(points[node], points[start_point])

    estimated_min_distance = min(haversine(points[node], points[i]) for i in unvisited)
    return estimated_min_distance + min(haversine(points[start_point], points[i]) for i in unvisited)

def a_star_tsp(points):
    start = 0
    visited = set([start])
    pq = [(0, start, [start], visited)]

    while pq:
        cost, node, path, visited = heapq.heappop(pq)

        if len(visited) == len(points):
            return path, cost + haversine(points[node], points[start])

        for i in range(len(points)):
            if i not in visited:
                new_cost = cost + haversine(points[node], points[i])
                new_visited = visited.copy()
                new_visited.add(i)
                new_path = path + [i]
                heapq.heappush(pq, (new_cost + heuristic(i, points, new_visited, start), i, new_path, new_visited))

    return [], 0  # Return an empty path and zero cost if no solution is found

# Qassim region cities GPS coordinates
points = [
    (26.32599, 43.97497),  # Buraidah
    (26.08427, 43.99355),  # Unaizah
    (25.86944, 43.49731),  # Ar Rass
    (26.14422, 43.65986),  # Al Bukayriyah
    (26.29945, 43.77493),  # Al Badayea
    (25.79886, 43.52471),  # Al Mithnab
    (26.12918, 43.64305),  # Al Rass
    (26.08748, 43.76251),  # An Nabhaniyah
    (26.07385, 44.30558),  # Badai
    (26.16556, 43.70222),  # Mulida
]

tour, total_distance = a_star_tsp(points)
print("Tour:", tour)
print("Total Distance (km):", total_distance)


Tour: [0, 4, 9, 3, 6, 2, 5, 7, 1, 8]
Total Distance (km): 222.08758743046752


### Naive `naive_nearest_neighbor_tsp` Function:

1. **Time Complexity**: The naive function generates all possible permutations of the points (excluding the starting point) to find the minimum path. If there are \( n \) points, the number of permutations is \( (n-1)! \). For each permutation, it calculates the total distance, which takes \( O(n) \) time. Thus, the time complexity is \( O(n \cdot (n-1)!) = ***O(n!)*** \).

2. **Space Complexity**: The function uses recursion to generate paths, and the maximum depth of the recursive call stack will be \( n \). It also stores the minimum path and distance, but the space is primarily dominated by the recursion stack and the space needed to store the paths. Thus, the space complexity is \( ***O(n)*** \).

### Optimized `optimized_nearest_neighbor_tsp` Function:

1. **Time Complexity**: For each of the \( n \) points, the function might need to check all other \( n-1 \) points to find the nearest unvisited neighbor, leading to an \( O(n^2) \) time complexity. However, because it uses a min heap (priority queue) to efficiently find the next closest point, each insertion and extraction operation is \( O(\log n) \), leading to an overall time complexity of \( ***O(n^2 \log n)*** \).

2. **Space Complexity**: The space complexity is dominated by the storage of the min heap, which contains at most \( n-1 \) elements, and the set to track visited nodes. Thus, the space complexity is \( ***O(n)*** \).



# **15 points as iput**

**Naïve version**